# Notebook for use on the Data Science Capstone Project

In [1]:
import pandas as pd
import numpy as np

In [2]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


## Week 3 Assignment: Segmenting and Clustering Neighborhoods in Toronto

##### Importing Toronto Suburbs Dataset from Wikipedia - using beautifulsoup

In [3]:
#install beautiful soup, geopy and folium libraries
!pip install bs4
!pip install geopy
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge folium=0.5.0 --yes

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1273 sha256=e7822dfa8a831157a7d307cd5af793adca552cec300553cf803b1cb2ba6021e9
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/0a/9e/ba/20e5bbc1afef3a491f0b3bb74d508f99403aabe76eda2167ca
Successfully built bs4
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/

In [4]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

#### Create variable (soup) to hold html data

In [5]:
T_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
data  = requests.get(T_url).text

In [6]:
soup = BeautifulSoup(data, 'html5lib')

In [6]:
#View the html structure for wikipedia page -- only used while testing code
#soup

#### Scraping data from wikipedia page - Appending to pandas dataframe

In [7]:
table_contents=[]
table = soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

#sort ascending by PostalCode
df.sort_values(by=['PostalCode'])

,PostalCode,Borough,Neighborhood
6,M1B,Scarborough,"Malvern, Rouge"
12,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
18,M1E,Scarborough,"Guildwood, Morningside, West Hill"
22,M1G,Scarborough,Woburn
26,M1H,Scarborough,Cedarbrae
...,...,...,...
64,M9N,York,Weston
70,M9P,Etobicoke,Westmount
77,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
89,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


The dataframe shape is:

In [8]:
df.shape

(103, 3)

In [9]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]))

The dataframe has 15 boroughs and 103 neighborhoods.


### Retreive Lat & Long data using Geospatial_Coordinates.csv

In [10]:
df_LL = pd.read_csv (r'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv')
print (df_LL)

#https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv

    Postal Code   Latitude  Longitude
0           M1B  43.806686 -79.194353
1           M1C  43.784535 -79.160497
2           M1E  43.763573 -79.188711
3           M1G  43.770992 -79.216917
4           M1H  43.773136 -79.239476
..          ...        ...        ...
98          M9N  43.706876 -79.518188
99          M9P  43.696319 -79.532242
100         M9R  43.688905 -79.554724
101         M9V  43.739416 -79.588437
102         M9W  43.706748 -79.594054

[103 rows x 3 columns]


### Rename Postal Code col.  Merge data on PostalCode field

In [11]:
df_LL.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
neighborhoods = pd.merge(df, df_LL, on='PostalCode', how='inner')
neighborhoods

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


## Import libraries to complete the 'Segmenting and Clustering Neighborhoods in Toronto' exercise

In [16]:
#!pip install geopy

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


In [12]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')


Solving environment: done

# All requested packages already installed.

Libraries imported.


### Identify Longitude & Latitude of Toronto

In [16]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


### Create a map of Toronto with neighborhoods superimposed

In [17]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Identify the Borough with the most postcodes.  Use this as the set for further analysis

In [18]:
neighborhoods['Borough'].value_counts()

North York                24
Scarborough               17
Downtown Toronto          17
Etobicoke                 11
Central Toronto            9
West Toronto               6
York                       5
East Toronto               4
East York                  4
East York/East Toronto     1
Queen's Park               1
East Toronto Business      1
Mississauga                1
Etobicoke Northwest        1
Downtown Toronto Stn A     1
Name: Borough, dtype: int64

North York chosen for Analysis.  Co-ordinates for North York are:

In [19]:
northyork_data = neighborhoods[neighborhoods['Borough'] == 'North York'].reset_index(drop=True)
northyork_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
3,M3B,North York,Don Mills North,43.745906,-79.352188
4,M6B,North York,Glencairn,43.709577,-79.445073


In [20]:
address = 'North York, Toronto'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of North York are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of North York are 43.7543263, -79.44911696639593.


### Create map of North York using latitude and longitude values

In [21]:
map_NorthYork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, label in zip(northyork_data['Latitude'], northyork_data['Longitude'], northyork_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_NorthYork)  
    
map_NorthYork

## Define Foursquare Credentials and Version

In [22]:
CLIENT_ID = 'P2MGPUXJG1RUBIVCB2OUICQTJ5WFL0QFN0EGDKE20ESPN1O2' # your Foursquare ID
CLIENT_SECRET = 'WCHHX33L0PD5QQERPYEUURZIITH4MAQE3ZXRS4O5AAWM3YSA' # your Foursquare Secret
VERSION = '20180604' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: P2MGPUXJG1RUBIVCB2OUICQTJ5WFL0QFN0EGDKE20ESPN1O2
CLIENT_SECRET:WCHHX33L0PD5QQERPYEUURZIITH4MAQE3ZXRS4O5AAWM3YSA


### Top 50 venues that are within a 300 meter radius of chosen neighborhood

First entry in the dataframe

In [23]:
northyork_data.loc[0, 'Neighborhood']

'Parkwoods'

Find co-ordinates of Parkwoods

In [24]:
neighborhood_latitude = northyork_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = northyork_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = northyork_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


### Top 50 Venues within 1000 meters

In [25]:
#create the GET request URL
LIMIT = 50
radius = 1000
CLIENT_ID = 'P2MGPUXJG1RUBIVCB2OUICQTJ5WFL0QFN0EGDKE20ESPN1O2' # your Foursquare ID
CLIENT_SECRET = 'WCHHX33L0PD5QQERPYEUURZIITH4MAQE3ZXRS4O5AAWM3YSA' # your Foursquare Secret
ACCESS_TOKEN = 'MH3GX20WHR1YYNVGXTNCHXHO2SZFQ3UOFKP5SQ3WMHNR1AIF' # your FourSquare Access Token
VERSION = '20180604'
url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        neighborhood_latitude, 
        neighborhood_longitude, 
        radius, 
        LIMIT)

url

'https://api.foursquare.com/v2/venues/explore?&client_id=P2MGPUXJG1RUBIVCB2OUICQTJ5WFL0QFN0EGDKE20ESPN1O2&client_secret=WCHHX33L0PD5QQERPYEUURZIITH4MAQE3ZXRS4O5AAWM3YSA&v=20180604&ll=43.7532586,-79.3296565&radius=1000&limit=50'

In [27]:
results = requests.get(url).json()

In [28]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4b8991cbf964a520814232e3',
  'name': "Allwyn's Bakery",
  'location': {'address': '81 Underhill drive',
   'lat': 43.75984035203157,
   'lng': -79.32471879917513,
   'labeledLatLngs': [{'label': 'display',
     'lat': 43.75984035203157,
     'lng': -79.32471879917513}],
   'distance': 833,
   'postalCode': 'M3A 1Z5',
   'cc': 'CA',
   'neighborhood': 'Parkwoods - Donalda',
   'city': 'Toronto',
   'state': 'ON',
   'country': 'Canada',
   'formattedAddress': ['81 Underhill drive', 'Toronto ON M3A 1Z5', 'Canada']},
  'categories': [{'id': '4bf58dd8d48988d144941735',
    'name': 'Caribbean Restaurant',
    'pluralName': 'Caribbean Restaurants',
    'shortName': 'Caribbean',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/caribbean_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'gro

In [29]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

clean the json and structure it into a pandas dataframe

In [30]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,Allwyn's Bakery,Caribbean Restaurant,43.759840,-79.324719
1,Brookbanks Park,Park,43.751976,-79.332140
2,Tim Hortons,Café,43.760668,-79.326368
3,Bruno's valu-mart,Grocery Store,43.746143,-79.324630
4,A&W,Fast Food Restaurant,43.760643,-79.326865
5,High Street Fish & Chips,Fish & Chips Shop,43.745260,-79.324949
6,Food Basics,Supermarket,43.760549,-79.326045
7,Shoppers Drug Mart,Pharmacy,43.745315,-79.325800
8,Shoppers Drug Mart,Pharmacy,43.760857,-79.324961
9,Variety Store,Food & Drink Shop,43.751974,-79.333114
